# Data Consistency Check

This notebook checks if the number of neighborhoods in the processed data matches the input data for each city, indicator, and year.

In [2]:
import json
import pandas as pd
from pathlib import Path
import requests
from io import StringIO
from collections import defaultdict

In [3]:
# Configure paths
BASE_DIR = Path.cwd().parent.parent
PROCESSED_FILES = {
    "barcelona": BASE_DIR / "data/processed/insert_ready_indicators_bcn.json",
    "madrid": BASE_DIR / "data/processed/insert_ready_indicators_madrid.json"
}

# Load files
with open(PROCESSED_FILES['barcelona']) as f_bcn, open(PROCESSED_FILES['madrid']) as f_mad:
    data_bcn = json.load(f_bcn)
    data_mad = json.load(f_mad)

# Convert to DataFrame
df_bcn = pd.DataFrame(data_bcn)
df_mad = pd.DataFrame(data_mad)

# Combine both
df_all = pd.concat([df_bcn, df_mad], ignore_index=True)

# Group and count unique geo_id by indicator_def_id, year, city_id
grouped = df_all.groupby(['indicator_def_id', 'year', 'city_id'])['geo_id'].nunique().reset_index()
grouped.rename(columns={'geo_id': 'unique_geo_ids'}, inplace=True)

# Show results
print(grouped)

    indicator_def_id  year  city_id  unique_geo_ids
0                  1  2019        1              73
1                  1  2020        1              73
2                  1  2020        2             131
3                  1  2021        1              73
4                  1  2021        2             131
5                  1  2022        1              73
6                  1  2022        2             131
7                  1  2023        1              73
8                  1  2023        2             131
9                  1  2024        2             131
10                 2  2019        1              73
11                 2  2020        1              73
12                 2  2020        2             131
13                 2  2021        1              73
14                 2  2021        2             131
15                 2  2022        2             131
16                 2  2023        2             131
17                 2  2024        2             131
18          

In [4]:
# Count records in each DataFrame
n_bcn = len(df_bcn)
n_mad = len(df_mad)
total = n_bcn + n_mad

print(f"Registers in Barcelona: {n_bcn}")
print(f"Registers in Madrid:    {n_mad}")
print(f"TOTAL to insert:    {total}")

Registers in Barcelona: 1095
Registers in Madrid:    1310
TOTAL to insert:    2405


In [5]:
n_indicators = df_all['indicator_def_id'].nunique()
n_neighborhoods = 73 + 131  # 204
n_districts = 10 + 21       # 31
n_cities = 2

expected_view_rows = n_indicators * (n_neighborhoods + n_districts + n_cities)

print(f"Unique indicators: {n_indicators}")
print(f"Expected rows in view: {expected_view_rows}")


Unique indicators: 4
Expected rows in view: 948


In [6]:
with open(PROCESSED_FILES['barcelona']) as f:
    indicators = json.load(f)

geo_ids = {i['geo_id'] for i in indicators}
print(f"Geo IDs in indicators: {sorted(geo_ids)}")

# Comparar con los geo_id reales
from supabase import create_client
import os
from dotenv import load_dotenv

load_dotenv()
supabase = create_client(os.getenv("SUPABASE_URL"), os.getenv("SUPABASE_SERVICE_KEY"))
res = supabase.table("neighbourhoods").select("id").eq("city_id", 1).execute()
neigh_ids = {n["id"] for n in res.data}
print(f"Missing in indicators: {sorted(neigh_ids - geo_ids)}")


Geo IDs in indicators: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73]
Missing in indicators: []


In [7]:
with open(PROCESSED_FILES['madrid']) as f:
    indicators = json.load(f)

geo_ids = {i['geo_id'] for i in indicators}
print(f"Geo IDs in indicators: {sorted(geo_ids)}")

# Comparar con los geo_id reales
from supabase import create_client
import os
from dotenv import load_dotenv

load_dotenv()
supabase = create_client(os.getenv("SUPABASE_URL"), os.getenv("SUPABASE_SERVICE_KEY"))
res = supabase.table("neighbourhoods").select("id").eq("city_id", 1).execute()
neigh_ids = {n["id"] for n in res.data}
print(f"Missing in indicators: {sorted(neigh_ids - geo_ids)}")


Geo IDs in indicators: [74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204]
Missing in indicators: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73]
